In [1]:
!git clone https://github.com/knazeri/edge-connect

Cloning into 'edge-connect'...
remote: Enumerating objects: 436, done.
remote: Total 436 (delta 0), reused 0 (delta 0), pack-reused 436
Receiving objects: 100% (436/436), 1.33 MiB | 1.08 MiB/s, done.
Resolving deltas: 100% (264/264), done.


In [160]:
%cd ./edge-connect/

/content/edge-connect


# Pretrained weight download
Weights of the model trained on Paris-Street View dataset can be downloaded [here](https://drive.google.com/drive/folders/1cGwDaZqDcqYU7kDuEbMXa9TP3uDJRBR1) and all of the 4 files must be kept in "psv" folder under edge-connect directory.

In [77]:
#Scipy 1.1.0 version is supported. You can also install through requirements file
!pip install scipy==1.1.0

##Test images preparation
After running the undergiven cell, manually upload the defective images and masks. Multiple Images and corresponding masks can be uploaded.

In [161]:
import os 
  
# Directory 
directoryI = "Images"
directoryM = "Masks"  
result= "results"
# Parent Directory path 
parent_dir = "/content/edge-connect"
  
# Path 
pathI = os.path.join(parent_dir, directoryI)
pathM = os.path.join(parent_dir, directoryM)
results = os.path.join(parent_dir, result)

# Create the directory 
os.mkdir(pathI)
os.mkdir(pathM) 
os.mkdir(results)

FileExistsError: ignored

##Main code

In [171]:
#The main file

import os
import cv2
import random
import numpy as np
import torch
import argparse
from shutil import copyfile
from src.config import Config
from src.edge_connect import EdgeConnect


def main(mode=None):
    
    config = load_config(mode)

    # cuda visble devices
    os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(str(e) for e in config.GPU)


    # init device
    if torch.cuda.is_available():
        config.DEVICE = torch.device("cuda")
        torch.backends.cudnn.benchmark = True   # cudnn auto-tuner
    else:
        config.DEVICE = torch.device("cpu")



    # set cv2 running threads to 1 (prevents deadlocks with pytorch dataloader)
    cv2.setNumThreads(0)


    # initialize random seed
    torch.manual_seed(config.SEED)
    torch.cuda.manual_seed_all(config.SEED)
    np.random.seed(config.SEED)
    random.seed(config.SEED)



    # build the model and initialize
    model = EdgeConnect(config)
    model.load()


    # model training
    if config.MODE == 1:
        config.print()
        print('\nstart training...\n')
        model.train()

    # model test
    elif config.MODE == 2:
        print('\nstart testing...\n')
        model.test()

    # eval mode
    else:
        print('\nstart eval...\n')
        model.eval()


def load_config(mode=None):
    config_path = os.path.join('./psv','config.yml')
    
    # create checkpoints path if does't exist
    if not os.path.exists('./psv'):
        os.makedirs('./psv')

    # copy config template if does't exist
    if not os.path.exists(config_path):
        copyfile('./config.yml.example', config_path)

    # load config file
    config = Config(config_path)

    # train mode
    if mode == 1:
        config.MODE = 1
        if args.model:
            config.MODEL = args.model

    # test mode
    elif mode == 2:
        config.MODE = 2
        config.MODEL = 3
        config.INPUT_SIZE = 0

        #if args.input is not None:
        config.TEST_FLIST = './Images'

        #if args.mask is not None:
        config.TEST_MASK_FLIST = './Masks'

        #if args.edge is not None:
        #    config.TEST_EDGE_FLIST = args.edge

        #if args.output is not None:
        config.RESULTS = './results'

    # eval mode
    elif mode == 3:
        config.MODE = 3
        config.MODEL = args.model if args.model is not None else 3

    return config

if __name__ == "__main__":
    main(mode=2)

Loading EdgeModel generator...
Loading InpaintingModel generator...

start testing...

1 image3.jpg

End test....


In [172]:
%cd ./results/

/content/edge-connect/results


##Post Processing Methods
Median filtering followed by weighted Gaussian filtering is performed on each image.

In [0]:
def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.0, threshold=0):
    """Return a sharpened version of the image, using an unsharp mask."""
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

In [174]:
import cv2
import numpy as np

list1= os.listdir('/content/edge-connect/results')
number= len(list1)
print(number)

path1= '/content/edge-connect/results/'

for i in range(1,number):
  image= list1[i]
  print(path1 + image)
  im= cv2.imread(path1 + image)
  final= cv2.medianBlur(im, 3)
  #cv2.imwrite(list1[i], final)
  sharpened_image = unsharp_mask(final)
  cv2.imwrite(image, sharpened_image)


2
/content/edge-connect/results/image3.jpg
